In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from pandas import Series

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf

In [ ]:
csv_path = 'inp/IBIRITE (ROLA MOCA)_MG.csv'

In [ ]:
data = pd.read_csv(csv_path, sep=';')
data

In [ ]:
data.rename(columns={"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "Chuva",
                     "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "$Pressão_{Inst}$",
                     "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "$Pressão_{Max}$",
                     "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "$Pressão_{Min}$",
                     "RADIACAO GLOBAL (KJ/m²)": "H",
                     "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "$Temperatura_{Inst}$",
                     "TEMPERATURA DO PONTO DE ORVALHO (°C)": "$Orvalho_{Inst}$",
                     "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "$Temperatura_{Max}$",
                     "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "$Temperatura_{Min}$",
                     "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "$Orvalho_{Max}$",
                     "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "$Orvalho_{Min}$",
                     "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "$Umidade_{Max}$",
                     "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "$Umidade_{Min}$",
                     "UMIDADE RELATIVA DO AR, HORARIA (%)": "$Umidade_{Inst}$",
                     "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "$Vento_{Dir}$",
                     "VENTO, RAJADA MAXIMA (m/s)": "$Vento_{Raj}$",
                     "VENTO, VELOCIDADE HORARIA (m/s)": "$Vento_{Vel}$",
                     "RADIACAO GLOBAL (Kj/m²)": "GLOBAL"}, inplace=True)
data

In [ ]:
data.set_index('DATE', inplace=True)
data.dropna(axis=0, how='all', subset=None, inplace=True)
data.dropna(axis=1, how='all', subset=None, inplace=True)

In [ ]:
#DEPENDENDO DO ANO E DA ESTAÇÃO OS VALORES DE RADIAÇÃO ESTÃO EM COLUNAS DIFERENTES
if 'GLOBAL' not in data.columns:
    data.rename(columns={"H": "GLOBAL"}, inplace=True)
if 'H' in data.columns:
    data['GLOBAL'].fillna(data['H'], inplace=True)
    data.drop(['H'], axis=1, inplace=True)

In [ ]:
train_dates = pd.to_datetime(data.index)
data = data[
    ['$Temperatura_{Inst}$', '$Temperatura_{Max}$', '$Temperatura_{Min}$', '$Umidade_{Max}$', '$Umidade_{Min}$',
     '$Umidade_{Inst}$', 'GLOBAL']]

cols = list(data)[0:7]
data = data[cols].astype(float)
data

In [ ]:
# GARANTINDO QUE TODOS OS DADOS ESTÃO SENDO TRATADOS COMO FLOAT
data = data.replace(',', '.', regex=True)
data['$Temperatura_{Inst}$'] = data['$Temperatura_{Inst}$'].astype(float)
data['$Temperatura_{Max}$'] = data['$Temperatura_{Max}$'].astype(float)
data['$Temperatura_{Min}$'] = data['$Temperatura_{Min}$'].astype(float)
data['GLOBAL'] = data['GLOBAL'].astype(float)

In [ ]:
#DELETANDO ALGUMA LEITURA ONDE O MÁXIMO É MENOR QUE O MÍNIMO
delIndex = data.loc[(data['$Temperatura_{Max}$'] < data['$Temperatura_{Min}$']) | (
        data['$Umidade_{Max}$'] < data['$Umidade_{Min}$'])].index
if len(delIndex) > 0:
    data.drop(delIndex, axis=0, inplace=True)

In [ ]:
#DELETANDO ALGUMA LEITURA ONDE O MÁXIMO É MENOR QUE O INSTANTÂNEO
delIndex = data.loc[(data['$Temperatura_{Max}$'] < data['$Temperatura_{Inst}$']) | (
        data['$Umidade_{Max}$'] < data['$Umidade_{Inst}$'])].index
if len(delIndex) > 0:
    data.drop(delIndex, axis=0, inplace=True)

In [ ]:
#DELETANDO ALGUMA LEITURA ONDE O INSTANTÂNEO É MENOR QUE O MÍNIMO
delIndex = data.loc[(data['$Temperatura_{Inst}$'] < data['$Temperatura_{Min}$']) | (
        data['$Umidade_{Inst}$'] < data['$Umidade_{Min}$'])].index
if len(delIndex) > 0:
    data.drop(delIndex, axis=0, inplace=True)

In [ ]:
# #MODIFICANDO A UNIDADE DA RADIAÇÃO DE (Kj/m²) PARA (Mj/m²)
data['GLOBAL'] = data['GLOBAL'].mul(0.001)

In [ ]:
#DELETANDO ALGUMA LINHA QUE AINDA TENHA NAN
data.dropna(inplace=True)

In [ ]:
# EXTRAINDO OS NOMES DAS COLUNAS
target_names = ['GLOBAL']
variable_names = data.columns.values
variable_names = np.delete(variable_names, np.where(variable_names == target_names))

In [ ]:
date_range = data.index

var_to_plot = data.columns
df = data[var_to_plot]

n = int(df.shape[0] * 0.7)
df.index = range(df.shape[0])
id0 = df.index <= n
id1 = df.index > n
print(data.index)

In [ ]:
X = data[variable_names]
y = data[target_names]

In [ ]:
# normalizing the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
data = scaler.fit_transform(data)


In [ ]:
print(X)

In [ ]:
from sklearn.model_selection import train_test_split

train_x = []
train_y = []

n_future = 1
n_past = 3

for i in range(n_past, len(data) - n_future + 1):
    train_x.append(data[i - n_past:i, 0:data.shape[1]])
    train_y.append(data[i + n_future - 1:i + n_future, 0])

train_x = np.array(train_x)
train_y = np.array(train_y)

# Separar os dados em conjuntos de treinamento e teste
test_size = 0.15
split_index = int(len(train_x) * (1 - test_size))

train_x, test_x = train_x[:split_index], train_x[split_index:]
train_y, test_y = train_y[:split_index], train_y[split_index:]


print("Shape de train_x:", train_x.shape)
print("Shape de train_y:", train_y.shape)
print("Shape de test_x:", test_x.shape)
print("Shape de test_y:", test_y.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import RootMeanSquaredError
from scikeras.wrappers import KerasRegressor
from evolutionary_search import EvolutionaryAlgorithmSearchCV

params = {
    'batch_size': [2048],
    'epochs': [100, 200],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.01, 0.001],
    'activation': ['relu', 'leaky_relu', 'segmoid'],
    'dropout': [0.1, 0.2],
    'neurons': [32, 64]
}


def create_model(learning_rate=0.1, optimizer='adam', activation='relu', dropout=0.1, neurons=1):
    model1 = Sequential()
    model1.add(CuDNNLSTM(neurons, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
    model1.add(CuDNNLSTM(int(neurons/2), return_sequences=False))
    model1.add(Dense(train_y.shape[1]))
    model1.add(Dropout(dropout))
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    model1.compile(loss=MeanSquaredError(), optimizer=optimizer,
                   metrics=[RootMeanSquaredError()])
    return model1


model = KerasRegressor(model=create_model, verbose=2, learning_rate=[0.01, 0.001], activation=['relu', 'leaky_relu', 'segmoid'],
                       dropout=[0.2, 0.4], neurons=[16, 32, 64])

checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

eas = EvolutionaryAlgorithmSearchCV(estimator=model,
                                    params=params,
                                    scoring='neg_mean_squared_error',
                                    cv=3,
                                    verbose=1,
                                    population_size=50,
                                    gene_mutation_prob=0.01,
                                    gene_crossover_prob=0.8,
                                    tournament_size=3,
                                    generations_number=15)

# Enable graph execution mode
tf.config.run_functions_eagerly(True)
# tf.config.run_functions_eagerly(False)

# Fit the model
eas.fit(train_x, train_y)

In [ ]:
print(eas.best_score_, eas.best_params_)

In [ ]:
predictions = eas.predict(test_x)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, max_error, median_absolute_error, mean_squared_log_error, explained_variance_score

mse = mean_squared_error(test_y, predictions)
r2 = r2_score(test_y, predictions)
mae = mean_absolute_error(test_y, predictions)
mape = mean_absolute_percentage_error(test_y, predictions)
me = max_error(test_y, predictions)
medae = median_absolute_error(test_y, predictions)
msle = mean_squared_log_error(test_y, predictions)
evs = explained_variance_score(test_y, predictions)

print('MSE: ', mse, '\nR2: ', r2, '\nMAE: ', mae, '\nMAPE: ', mape, '\nME: ', me, '\nMEDAE: ', medae, '\nMSLE: ', msle, '\nEVS: ', evs)

In [ ]:
# inverte a escala para os valores originais
predictions_copies = np.repeat(predictions, train_x.shape[2], axis=-1)
original = scaler.inverse_transform(predictions_copies)[:,0]
print(original)

test_y_copies = np.repeat(test_y, train_x.shape[2], axis=-1)
original_test = scaler.inverse_transform(test_y_copies)[:,0]
print(original_test)


In [ ]:
print(train_dates.shape)
print(original.shape)
print(original_test.shape)

# Plotar os resultados
pl.plot(original, label='Predição')
pl.plot(original_test, label='Dados Originais')
pl.legend()
pl.xlabel('Índice')
pl.ylabel('Valor')
pl.title('Dados Predição vs. Dados de Teste')
pl.show()